In [48]:
import pandas as pd


In [1]:
import re

In [7]:
print(re.sub('[^A-Za-z0-9 $]+', '', '\tHey\n you 2312 R$!'))

Hey you 2312 R$


In [5]:
print('\tHey\n you 2312 R$!')

	Hey
 you 2312 R$!


In [49]:
data = pd.read_csv('../data/tceTextData.csv')

In [50]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 324728 entries, 0 to 324727
Data columns (total 2 columns):
 #   Column                       Non-Null Count   Dtype 
---  ------                       --------------   ----- 
 0   Empenho (Histórico)(EOF)     324728 non-null  object
 1   Natureza Despesa (Cod)(EOF)  324728 non-null  object
dtypes: object(2)
memory usage: 5.0+ MB


In [51]:
data['Natureza Despesa (Cod)(EOF)'].value_counts()

3.1.90.11.10    6203
3.3.90.18.05    6130
3.3.90.92.23    5673
3.3.90.30.09    5356
3.3.90.93.02    5284
                ... 
4.4.90.40.16       1
3.3.90.39.72       1
3.1.90.05.02       1
3.3.90.46.03       1
4.4.90.52.32       1
Name: Natureza Despesa (Cod)(EOF), Length: 650, dtype: int64

In [52]:
import re
import string 
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from unidecode import unidecode

def tolower(text:str):
    return text.lower()

def remove_urls(text):
    temp = re.sub("http.?://[^\s]+[\s]?", "", text)
    text = re.sub("(www\.)[^\s]+[\s]?", "", temp)
    return text

def remove_digits(text:str):
    return re.sub(r'\d+', '', text)

def remove_punctuation(text:str):
    punct = string.punctuation
    translation_table = str.maketrans(punct, len(punct)*' ')
    return text.translate(translation_table)

def remove_stopwords(text:str):
    stopwords_list = stopwords.words('portuguese')
    return ' '.join([word for word in text.split(' ') if (word not in stopwords_list) and len(word) > 1])

def lemmatize(text:str):
    lemmatizer = WordNetLemmatizer()
    return ' '.join([lemmatizer.lemmatize(word) for word in text.split(' ')])

def stemming(text:str):
    stemmer = PorterStemmer()
    return ' '.join([stemmer.stem(word) for word in text.split(' ')])

def fix_column_name(text:str):
    text = re.sub("(\(eof\))|\(|\)", "", text) 
    text = re.sub("(_codigo/nome)|(_cod/nome)|(_dia/mes/ano)|", "", text)
    text = re.sub(' ', '', text)
    return unidecode(text.lower())

def tfidfPipeline(text):
    return stemming(remove_stopwords(remove_punctuation(remove_digits(remove_urls(tolower(unidecode(text)))))))

def embeddingPipeline(text):
    return remove_punctuation(remove_digits(remove_urls(tolower(unidecode(text)))))



In [53]:
# fix column names
data.columns = list(map(fix_column_name, data.columns))

In [54]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 324728 entries, 0 to 324727
Data columns (total 2 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   empenhohistoricoeof    324728 non-null  object
 1   naturezadespesacodeof  324728 non-null  object
dtypes: object(2)
memory usage: 5.0+ MB


In [55]:
data.empenhohistoricoeof = data.empenhohistoricoeof.map(tfidfPipeline)


In [56]:
data.empenhohistoricoeof

0                       referent construcao nova sede alego
1                 referent folha pagamento estagiario marco
2                           referent folha estagiario abril
3                            referent folha estagiario maio
4                           referent folha estagiario junho
                                ...                        
324723    importancia empenha referent contrato celebrad...
324724    importancia empenha referent contrato celebrad...
324725    importancia empenha referent contrato prestaca...
324726    importancia empenha referent contrato celebrad...
324727    importancia empenha referent contrato celebrad...
Name: empenhohistoricoeof, Length: 324728, dtype: object

In [57]:
data.to_csv('../data/tfidfReadyTextData.csv', index=False)

In [58]:
data = pd.read_csv('../data/tceTextData.csv')
data.columns = list(map(fix_column_name, data.columns))
data.empenhohistoricoeof = data.empenhohistoricoeof.map(embeddingPipeline)
data.to_csv('../data/embeddingReadyTextData.csv', index=False)